In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision as tv
import matplotlib.pyplot as plt
import PIL
import os
import my_utils as mu
from shutil import copyfile

In [2]:
custom_transform = tv.transforms.Compose([
    tv.transforms.Resize((224,224)), #expected to be at least 224 in order to wrok with pretrained models
    tv.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) #for feature extractor
])

In [3]:
class BasicNet(nn.Module): #redefining since one can't easily import from an ipynb notebook 
    '''
    This network will take output from the feature extractor and hopefully do a good job at classifying the images.
    '''
    def __init__(self):
        super(BasicNet,self).__init__()
        self.dense1 = nn.Linear(1000,16)
        self.dense2 = nn.Linear(16,1)
        self.act1 = nn.LeakyReLU()
        self.act2 = nn.Sigmoid()
    def forward(self,x):
        x = feature_extractor(x)
        x = torch.flatten(x)
        x = self.dense1(x)
        x = self.act1(x)
        x = self.dense2(x)
        x = self.act2(x)
        return x

In [4]:
feature_extractor = tv.models.squeezenet1_1(pretrained=True).to('cuda')
net = BasicNet().to('cuda')
sd_path = './training_run_data/best_state_dict.pkl'
sd = torch.load(sd_path)
net.load_state_dict(sd)

<All keys matched successfully>

In [38]:
path_to_images = './photos_to_classify/' #folder full of images to classify
people_path = './people_folder/' #this is folder where images with people will go
no_people_path = './no_people_folder/' #make sure these paths end with '/'
f_list = os.listdir(path_to_images)
for i in range(len(f_list)):
    img = tv.io.read_image(path_to_images + f_list[i])
    s = img.size()
    if s[0] < 3: #if image is greyscale, make sure dims are correct.
        img = img.expand(3,s[1],s[2]) 
    img = img.unsqueeze(0) #adding dim so network can operate on it
    
    #convert from uint8 to 0-1
    img = img.type('torch.FloatTensor')
    img = img / 255 #get in range 0-1
    img = custom_transform(img) #normalize
    out = net(img.to('cuda'))
    out = float(out)
    
    #copy image to appropriate folder
    src = path_to_images + f_list[i]
    if out < 0.5: 
        dst = people_path + f_list[i]
    else:
        dst = no_people_path + f_list[i]
    copyfile(src,dst)